In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Extract python codes from a file

In [2]:
import os

file_dir = '../../gpt-discord-bot/src/'
file_list = os.listdir(file_dir); file_list

['base.py',
 'completion.py',
 'config.yaml',
 'constants.py',
 'main.py',
 'moderation.py',
 'utils.py']

In [3]:
fname = os.path.join(file_dir, file_list[0]); fname
f = open(fname, "r")
code = f.read()
code

'../../gpt-discord-bot/src/base.py'

'from dataclasses import dataclass\nfrom typing import Optional, List\n\nSEPARATOR_TOKEN = "<|endoftext|>"\n\n\n@dataclass(frozen=True)\nclass Message:\n    user: str\n    text: Optional[str] = None\n\n    def render(self):\n        result = self.user + ":"\n        if self.text is not None:\n            result += " " + self.text\n        return result\n\n\n@dataclass\nclass Conversation:\n    messages: List[Message]\n\n    def prepend(self, message: Message):\n        self.messages.insert(0, message)\n        return self\n\n    def render(self):\n        return f"\\n{SEPARATOR_TOKEN}".join(\n            [message.render() for message in self.messages]\n        )\n\n\n@dataclass(frozen=True)\nclass Config:\n    name: str\n    instructions: str\n    example_conversations: List[Conversation]\n\n\n@dataclass(frozen=True)\nclass Prompt:\n    header: Message\n    examples: List[Conversation]\n    convo: Conversation\n\n    def render(self):\n        return f"\\n{SEPARATOR_TOKEN}".join(\n    

### Add prompt to code to extract `class` and `def`

In [4]:
prompt_postfix = """ 
###
Please make a table summarizing the type of function in the code above:
|type|name|
|class|Message|
|def| render|
"""

In [5]:
prompt = code +  prompt_postfix
prompt

'from dataclasses import dataclass\nfrom typing import Optional, List\n\nSEPARATOR_TOKEN = "<|endoftext|>"\n\n\n@dataclass(frozen=True)\nclass Message:\n    user: str\n    text: Optional[str] = None\n\n    def render(self):\n        result = self.user + ":"\n        if self.text is not None:\n            result += " " + self.text\n        return result\n\n\n@dataclass\nclass Conversation:\n    messages: List[Message]\n\n    def prepend(self, message: Message):\n        self.messages.insert(0, message)\n        return self\n\n    def render(self):\n        return f"\\n{SEPARATOR_TOKEN}".join(\n            [message.render() for message in self.messages]\n        )\n\n\n@dataclass(frozen=True)\nclass Config:\n    name: str\n    instructions: str\n    example_conversations: List[Conversation]\n\n\n@dataclass(frozen=True)\nclass Prompt:\n    header: Message\n    examples: List[Conversation]\n    convo: Conversation\n\n    def render(self):\n        return f"\\n{SEPARATOR_TOKEN}".join(\n    

### Request a response from API

In [6]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_type = "azure"
openai.api_base = "https://tutorial-openai-01-2023.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

True

In [19]:
def prompt_api(engine, prompt, temperature, max_tokens, top_p, frequency_penalty, presence_penalty, best_of, stop):
  response = openai.Completion.create(
    engine=engine,
    prompt=prompt,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty, 
    presence_penalty=presence_penalty,
    best_of=best_of,
    stop=stop)
  return(response)

def print_response(response):
  response_text = response['choices'][0]['text']
  return response_text


In [21]:
engine="text-davinci-003"
prompt=prompt
temperature=0 # [0, 1]
max_tokens=100
top_p=1
frequency_penalty=0 # [0, 2]
presence_penalty=0 # [0, 2]
best_of=1
stop=["###"]

response = prompt_api(engine, prompt, temperature, max_tokens, top_p, frequency_penalty, presence_penalty, best_of, stop)
print(prompt_postfix)
response_text = print_response(response); print(response_text)

 
###
Please make a table summarizing the type of function in the code above:
|type|name|
|class|Message|
|def| render|

|dataclass|Conversation|
|dataclass|Config|
|dataclass|Prompt|
|def|prepend|
|def|render|


In [24]:
output = prompt_postfix + response_text
print(output)

 
###
Please make a table summarizing the type of function in the code above:
|type|name|
|class|Message|
|def| render|
|dataclass|Conversation|
|dataclass|Config|
|dataclass|Prompt|
|def|prepend|
|def|render|


### Save output to file

In [32]:
output_file_dir = "../output/extract_entities/"
fname = os.path.join(output_file_dir, file_list[0] + '.output'); fname
output_file = open(fname, "w")
output_file.write(output)
output_file.close()

'../output/extract_entities/base.py.output'

209